In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from torchvision import models
from sklearn.preprocessing import LabelEncoder
import os

# 设置随机种子以确保可重复性
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# 数据集类
class NPYDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        try:
            self.annotations = pd.read_csv(csv_file, encoding='utf-8')
        except UnicodeDecodeError:
            self.annotations = pd.read_csv(csv_file, encoding='gbk')
        self.root_dir = root_dir
        self.le = LabelEncoder()
        self.annotations['labels'] = self.annotations['labels'].apply(lambda x: x.strip("[]'"))
        self.annotations['labels'] = self.le.fit_transform(self.annotations['labels'])

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, -1]) + '.npy')
        image = np.load(img_name)
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = self.annotations.iloc[index, 2]
        file_name = os.path.basename(img_name)  # 获取文件名
        return image, label, file_name  # 返回图像、标签和文件名

# 模型定义
class ResNet2D(nn.Module):
    def __init__(self, num_classes):
        super(ResNet2D, self).__init__()
        # 使用ResNet50作为基础模型
        self.resnet = models.resnet50(pretrained=True)
        # 冻结预训练模型的参数
        for param in self.resnet.parameters():
            param.requires_grad = False

        # 替换全连接层以适应新的分类任务
        num_ftrs = self.resnet.fc.in_features  # 获取全连接层的输入特征数
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)  # 替换全连接层

    def forward(self, x):
        x = self.resnet(x)
        return x

# 之后的优化器、学习率调度器和训练循环可以保持不变

# 训练和评估参数
num_epochs = 10
batch_size = 32
learning_rate = 0.001

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 数据加载
train_dataset = NPYDataset(csv_file=r'C:\Users\HP\Desktop\指南者\项目\sci\Updated_D_N_train.csv', root_dir=r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_gaus\normalized_gaus\normalized_gaus_train_images')
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = NPYDataset(csv_file=r'C:\Users\HP\Desktop\指南者\项目\sci\Updated_D_N_test.csv', root_dir=r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_gaus\normalized_gaus\normalized_gaus_test_images')
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

validation_dataset = NPYDataset(csv_file=r'C:\Users\HP\Desktop\指南者\项目\sci\Updated_D_N_validation.csv', root_dir=r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_gaus\normalized_gaus\normalized_gaus_validation_images')
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=False)

# 模型初始化
num_classes = len(np.unique(train_dataset.annotations['labels']))

# 创建一个权重数组
# 这里假设类别标签已经编码为0, 1
weights = torch.tensor([1.0, 1.0], dtype=torch.float32).to(device)

# 使用加权损失函数
criterion = nn.CrossEntropyLoss(weight=weights)

model = ResNet2D(num_classes=num_classes).to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-3)  # 增加权重衰减

# 学习率调度器
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# 训练模型的代码...
# 请根据您的具体需求添加训练循环和验证/测试循环

In [6]:
model_path = r'C:\Users\HP\Desktop\指南者\项目\眼病预测\gaus_D_N classification\trained_model.pth'  # 指定模型保存路径

In [7]:
# 创建与之前相同的模型实例
# 请将num_classes替换为您实际使用的类别数
model = ResNet2D(num_classes=2)

# 加载保存的模型状态字典，确保映射到CPU
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# 如果你要继续在训练模式下使用模型，别忘了调用 model.train()
# 如果是用于评估或预测，调用 model.eval()

<All keys matched successfully>

In [12]:
import os
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import matplotlib.pyplot as plt
import torch

# 假设你的模型、test_loader以及device已经定义好
# 定义目标层
target_layers = [model.resnet.layer4[-1]]  # ResNet50的最后一个卷积层

# 创建Grad-CAM对象
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available())

# 指定保存目录
save_dir = r'C:\Users\HP\Desktop\指南者\项目\sci\gaus_ResNet50_grad_cam'
os.makedirs(save_dir, exist_ok=True)  # 创建目录（如果目录不存在）

model.eval()  # 确保模型在评估模式
with torch.enable_grad():  # 确保可以计算梯度
    for images, labels, file_names in test_loader:  # 现在也接收文件名
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        for index in range(images.size(0)):
            input_tensor = images[index].unsqueeze(0)  # 获取输入张量
            input_tensor.requires_grad_(True)

            # 生成Grad-CAM热图
            grayscale_cam = cam(input_tensor=input_tensor, targets=None)[0, :]
            img = images[index].permute(1, 2, 0).cpu().numpy()
            img = (img - img.min()) / (img.max() - img.min())  # 归一化图像
            visualization = show_cam_on_image(img, grayscale_cam, use_rgb=True)
            
            # 保存Grad-CAM热图
            cam_fig_path = os.path.join(save_dir, f'{file_names[index]}_gradcam.png')
            plt.imsave(cam_fig_path, visualization)
            
            print(f'Saved Grad-CAM image for {file_names[index]}')


Saved Grad-CAM image for 4709.npy
Saved Grad-CAM image for 1723.npy
Saved Grad-CAM image for 4126.npy
Saved Grad-CAM image for 3821.npy
Saved Grad-CAM image for 4488.npy
Saved Grad-CAM image for 6363.npy
Saved Grad-CAM image for 5613.npy
Saved Grad-CAM image for 5823.npy
Saved Grad-CAM image for 5072.npy
Saved Grad-CAM image for 4009.npy
Saved Grad-CAM image for 5968.npy
Saved Grad-CAM image for 964.npy
Saved Grad-CAM image for 5756.npy
Saved Grad-CAM image for 5697.npy
Saved Grad-CAM image for 804.npy
Saved Grad-CAM image for 3346.npy
Saved Grad-CAM image for 14.npy
Saved Grad-CAM image for 563.npy
Saved Grad-CAM image for 3917.npy
Saved Grad-CAM image for 4095.npy
Saved Grad-CAM image for 4696.npy
Saved Grad-CAM image for 848.npy
Saved Grad-CAM image for 4122.npy
Saved Grad-CAM image for 5332.npy
Saved Grad-CAM image for 668.npy
Saved Grad-CAM image for 5313.npy
Saved Grad-CAM image for 6093.npy
Saved Grad-CAM image for 6345.npy
Saved Grad-CAM image for 1215.npy
Saved Grad-CAM image 

## 血管分割

In [13]:
import numpy as np
import os
from pathlib import Path
from PIL import Image

# 函数用于归一化图片
def normalize_image(image_path, output_folder):
    # 读取图片并转换为浮点数
    image_array = np.array(Image.open(image_path), dtype=np.float32)
    
    # 计算均值和标准差
    mean = image_array.mean()
    std_dev = image_array.std()

    # 归一化处理
    normalized_image_array = (image_array - mean) / std_dev

    # 确保输出目录存在
    output_path = Path(output_folder)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # 保存归一化后的图像为.npy文件
    np.save(output_path / (Path(image_path).stem + '.npy'), normalized_image_array)

# 归一化指定文件夹中的所有图片
def normalize_folder(folder_path, output_folder):
    # 获取所有JPEG图片的文件路径
    image_paths = list(Path(folder_path).glob('*.jpg'))
    for image_path in image_paths:
        normalize_image(image_path, output_folder)

# 调用函数归一化train_images和test_images文件夹中的图片
normalize_folder(r'C:\Users\HP\Desktop\指南者\项目\sci\chanel2\D_N_chosen_segmentation\predicted_train', r'C:\Users\HP\Desktop\指南者\项目\sci\chanel2\D_N_chosen_segmentation\normalized_train_images')
normalize_folder(r'C:\Users\HP\Desktop\指南者\项目\sci\chanel2\D_N_chosen_segmentation\predicted_test', r'C:\Users\HP\Desktop\指南者\项目\sci\chanel2\D_N_chosen_segmentation\normalized_test_images')
normalize_folder(r'C:\Users\HP\Desktop\指南者\项目\sci\chanel2\D_N_chosen_segmentation\predicted_validation', r'C:\Users\HP\Desktop\指南者\项目\sci\chanel2\D_N_chosen_segmentation\normalized_validation_images')

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from torchvision import models
from sklearn.preprocessing import LabelEncoder
import os

# 设置随机种子以确保可重复性
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# 数据集类
class NPYDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        try:
            self.annotations = pd.read_csv(csv_file, encoding='utf-8')
        except UnicodeDecodeError:
            self.annotations = pd.read_csv(csv_file, encoding='gbk')
        self.root_dir = root_dir
        self.le = LabelEncoder()
        self.annotations['labels'] = self.annotations['labels'].apply(lambda x: x.strip("[]'"))
        self.annotations['labels'] = self.le.fit_transform(self.annotations['labels'])

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, -1]) + '.npy')
        image = np.load(img_name)
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = self.annotations.iloc[index, 2]
        file_name = os.path.basename(img_name)  # 获取文件名
        return image, label, file_name  # 返回图像、标签和文件名

# 模型定义
class ResNet2D(nn.Module):
    def __init__(self, num_classes):
        super(ResNet2D, self).__init__()
        # 使用ResNet50作为基础模型
        self.resnet = models.resnet50(pretrained=True)
        # 冻结预训练模型的参数
        for param in self.resnet.parameters():
            param.requires_grad = False

        # 替换全连接层以适应新的分类任务
        num_ftrs = self.resnet.fc.in_features  # 获取全连接层的输入特征数
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)  # 替换全连接层

    def forward(self, x):
        x = self.resnet(x)
        return x

# 之后的优化器、学习率调度器和训练循环可以保持不变

# 训练和评估参数
num_epochs = 10
batch_size = 32
learning_rate = 0.001

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 数据加载
train_dataset = NPYDataset(csv_file=r'C:\Users\HP\Desktop\指南者\项目\sci\Updated_D_N_train.csv', root_dir=r'C:\Users\HP\Desktop\指南者\项目\sci\chanel2\D_N_chosen_segmentation\normalized_train_images')
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = NPYDataset(csv_file=r'C:\Users\HP\Desktop\指南者\项目\sci\Updated_D_N_test.csv', root_dir=r'C:\Users\HP\Desktop\指南者\项目\sci\chanel2\D_N_chosen_segmentation\normalized_test_images')
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

validation_dataset = NPYDataset(csv_file=r'C:\Users\HP\Desktop\指南者\项目\sci\Updated_D_N_validation.csv', root_dir=r'C:\Users\HP\Desktop\指南者\项目\sci\chanel2\D_N_chosen_segmentation\normalized_validation_images')
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=False)

# 模型初始化
num_classes = len(np.unique(train_dataset.annotations['labels']))

# 创建一个权重数组
# 这里假设类别标签已经编码为0, 1
weights = torch.tensor([1.0, 1.0], dtype=torch.float32).to(device)

# 使用加权损失函数
criterion = nn.CrossEntropyLoss(weight=weights)

model = ResNet2D(num_classes=num_classes).to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-3)  # 增加权重衰减

# 学习率调度器
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# 训练模型的代码...
# 请根据您的具体需求添加训练循环和验证/测试循环

C:\Users\HP\Anaconda3\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
C:\Users\HP\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
model_path = r'C:\Users\HP\Desktop\指南者\项目\眼病预测\gaus_D_N classification\trained_model.pth'  # 指定模型保存路径

In [16]:
# 创建与之前相同的模型实例
# 请将num_classes替换为您实际使用的类别数
model = ResNet2D(num_classes=2)

# 加载保存的模型状态字典，确保映射到CPU
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

# 如果你要继续在训练模式下使用模型，别忘了调用 model.train()
# 如果是用于评估或预测，调用 model.eval()

<All keys matched successfully>

In [18]:
import os
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import matplotlib.pyplot as plt
import torch

# 假设你的模型、test_loader以及device已经定义好
# 定义目标层
target_layers = [model.resnet.layer4[-1]]  # ResNet50的最后一个卷积层

# 创建Grad-CAM对象
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available())

# 指定保存目录
save_dir = r'C:\Users\HP\Desktop\指南者\项目\sci\seg_ResNet50_grad_cam'
os.makedirs(save_dir, exist_ok=True)  # 创建目录（如果目录不存在）

model.eval()  # 确保模型在评估模式
with torch.enable_grad():  # 确保可以计算梯度
    for images, labels, file_names in test_loader:  # 现在也接收文件名
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        for index in range(images.size(0)):
            input_tensor = images[index].unsqueeze(0)  # 获取输入张量
            input_tensor.requires_grad_(True)

            # 生成Grad-CAM热图
            grayscale_cam = cam(input_tensor=input_tensor, targets=None)[0, :]
            img = images[index].permute(1, 2, 0).cpu().numpy()
            img = (img - img.min()) / (img.max() - img.min())  # 归一化图像
            visualization = show_cam_on_image(img, grayscale_cam, use_rgb=True)
            
            # 保存Grad-CAM热图
            cam_fig_path = os.path.join(save_dir, f'{file_names[index]}_gradcam.png')
            plt.imsave(cam_fig_path, visualization)
            
            print(f'Saved Grad-CAM image for {file_names[index]}')


Saved Grad-CAM image for 4709.npy
Saved Grad-CAM image for 1723.npy
Saved Grad-CAM image for 4126.npy
Saved Grad-CAM image for 3821.npy
Saved Grad-CAM image for 4488.npy
Saved Grad-CAM image for 6363.npy
Saved Grad-CAM image for 5613.npy
Saved Grad-CAM image for 5823.npy
Saved Grad-CAM image for 5072.npy
Saved Grad-CAM image for 4009.npy
Saved Grad-CAM image for 5968.npy
Saved Grad-CAM image for 964.npy
Saved Grad-CAM image for 5756.npy
Saved Grad-CAM image for 5697.npy
Saved Grad-CAM image for 804.npy
Saved Grad-CAM image for 3346.npy
Saved Grad-CAM image for 14.npy
Saved Grad-CAM image for 563.npy
Saved Grad-CAM image for 3917.npy
Saved Grad-CAM image for 4095.npy
Saved Grad-CAM image for 4696.npy
Saved Grad-CAM image for 848.npy
Saved Grad-CAM image for 4122.npy
Saved Grad-CAM image for 5332.npy
Saved Grad-CAM image for 668.npy
Saved Grad-CAM image for 5313.npy
Saved Grad-CAM image for 6093.npy
Saved Grad-CAM image for 6345.npy
Saved Grad-CAM image for 1215.npy
Saved Grad-CAM image 

## 原图

In [19]:
import numpy as np
import os
from pathlib import Path
from PIL import Image

# 函数用于归一化图片
def normalize_image(image_path, output_folder):
    # 读取图片并转换为浮点数
    image_array = np.array(Image.open(image_path), dtype=np.float32)
    
    # 计算均值和标准差
    mean = image_array.mean()
    std_dev = image_array.std()

    # 归一化处理
    normalized_image_array = (image_array - mean) / std_dev

    # 确保输出目录存在
    output_path = Path(output_folder)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # 保存归一化后的图像为.npy文件
    np.save(output_path / (Path(image_path).stem + '.npy'), normalized_image_array)

# 归一化指定文件夹中的所有图片
def normalize_folder(folder_path, output_folder):
    # 获取所有JPEG图片的文件路径
    image_paths = list(Path(folder_path).glob('*.jpg'))
    for image_path in image_paths:
        normalize_image(image_path, output_folder)

# 调用函数归一化train_images和test_images文件夹中的图片
normalize_folder(r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_raw\D_N_train_images', r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_raw\normalized_train_images')
normalize_folder(r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_raw\D_N_test_images', r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_raw\normalized_test_images')
normalize_folder(r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_raw\D_N_validation_images', r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_raw\normalized_validation_images')

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from torchvision import models
from sklearn.preprocessing import LabelEncoder
import os

# 设置随机种子以确保可重复性
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# 数据集类
class NPYDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        try:
            self.annotations = pd.read_csv(csv_file, encoding='utf-8')
        except UnicodeDecodeError:
            self.annotations = pd.read_csv(csv_file, encoding='gbk')
        self.root_dir = root_dir
        self.le = LabelEncoder()
        self.annotations['labels'] = self.annotations['labels'].apply(lambda x: x.strip("[]'"))
        self.annotations['labels'] = self.le.fit_transform(self.annotations['labels'])

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[index, -1]) + '.npy')
        image = np.load(img_name)
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)
        label = self.annotations.iloc[index, 2]
        file_name = os.path.basename(img_name)  # 获取文件名
        return image, label, file_name  # 返回图像、标签和文件名

# 模型定义
class ResNet2D(nn.Module):
    def __init__(self, num_classes):
        super(ResNet2D, self).__init__()
        # 使用ResNet50作为基础模型
        self.resnet = models.resnet50(pretrained=True)
        # 冻结预训练模型的参数
        for param in self.resnet.parameters():
            param.requires_grad = False

        # 替换全连接层以适应新的分类任务
        num_ftrs = self.resnet.fc.in_features  # 获取全连接层的输入特征数
        self.resnet.fc = nn.Linear(num_ftrs, num_classes)  # 替换全连接层

    def forward(self, x):
        x = self.resnet(x)
        return x

# 之后的优化器、学习率调度器和训练循环可以保持不变

# 训练和评估参数
num_epochs = 10
batch_size = 32
learning_rate = 0.001

# 设备配置
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 数据加载
train_dataset = NPYDataset(csv_file=r'C:\Users\HP\Desktop\指南者\项目\sci\Updated_D_N_train.csv', root_dir=r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_raw\normalized_train_images')
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = NPYDataset(csv_file=r'C:\Users\HP\Desktop\指南者\项目\sci\Updated_D_N_test.csv', root_dir=r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_raw\normalized_test_images')
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

validation_dataset = NPYDataset(csv_file=r'C:\Users\HP\Desktop\指南者\项目\sci\Updated_D_N_validation.csv', root_dir=r'C:\Users\HP\Desktop\指南者\项目\sci\chanel1\D_N_raw\normalized_validation_images')
validation_loader = DataLoader(dataset=validation_dataset, batch_size=batch_size, shuffle=False)

# 模型初始化
num_classes = len(np.unique(train_dataset.annotations['labels']))

# 创建一个权重数组
# 这里假设类别标签已经编码为0, 1
weights = torch.tensor([1.0, 1.0], dtype=torch.float32).to(device)

# 使用加权损失函数
criterion = nn.CrossEntropyLoss(weight=weights)

model = ResNet2D(num_classes=num_classes).to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-3)  # 增加权重衰减

# 学习率调度器
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# 训练模型的代码...
# 请根据您的具体需求添加训练循环和验证/测试循环

In [23]:
import os
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
import matplotlib.pyplot as plt
import torch

# 假设你的模型、test_loader以及device已经定义好
# 定义目标层
target_layers = [model.resnet.layer4[-1]]  # ResNet50的最后一个卷积层

# 创建Grad-CAM对象
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available())

# 指定保存目录
save_dir = r'C:\Users\HP\Desktop\指南者\项目\sci\raw_ResNet50_grad_cam'
os.makedirs(save_dir, exist_ok=True)  # 创建目录（如果目录不存在）

model.eval()  # 确保模型在评估模式
with torch.enable_grad():  # 确保可以计算梯度
    for images, labels, file_names in test_loader:  # 现在也接收文件名
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        for index in range(images.size(0)):
            input_tensor = images[index].unsqueeze(0)  # 获取输入张量
            input_tensor.requires_grad_(True)

            # 生成Grad-CAM热图
            grayscale_cam = cam(input_tensor=input_tensor, targets=None)[0, :]
            img = images[index].permute(1, 2, 0).cpu().numpy()
            img = (img - img.min()) / (img.max() - img.min())  # 归一化图像
            visualization = show_cam_on_image(img, grayscale_cam, use_rgb=True)
            
            # 保存Grad-CAM热图
            cam_fig_path = os.path.join(save_dir, f'{file_names[index]}_gradcam.png')
            plt.imsave(cam_fig_path, visualization)
            
            print(f'Saved Grad-CAM image for {file_names[index]}')


Saved Grad-CAM image for 4709.npy
Saved Grad-CAM image for 1723.npy
Saved Grad-CAM image for 4126.npy
Saved Grad-CAM image for 3821.npy
Saved Grad-CAM image for 4488.npy
Saved Grad-CAM image for 6363.npy
Saved Grad-CAM image for 5613.npy
Saved Grad-CAM image for 5823.npy
Saved Grad-CAM image for 5072.npy
Saved Grad-CAM image for 4009.npy
Saved Grad-CAM image for 5968.npy
Saved Grad-CAM image for 964.npy
Saved Grad-CAM image for 5756.npy
Saved Grad-CAM image for 5697.npy
Saved Grad-CAM image for 804.npy
Saved Grad-CAM image for 3346.npy
Saved Grad-CAM image for 14.npy
Saved Grad-CAM image for 563.npy
Saved Grad-CAM image for 3917.npy
Saved Grad-CAM image for 4095.npy
Saved Grad-CAM image for 4696.npy
Saved Grad-CAM image for 848.npy
Saved Grad-CAM image for 4122.npy
Saved Grad-CAM image for 5332.npy
Saved Grad-CAM image for 668.npy
Saved Grad-CAM image for 5313.npy
Saved Grad-CAM image for 6093.npy
Saved Grad-CAM image for 6345.npy
Saved Grad-CAM image for 1215.npy
Saved Grad-CAM image 